In [69]:
import os
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind, levene, chi2_contingency, mannwhitneyu
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from scipy import stats
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.model_selection import StratifiedKFold


def contin_feature(data_1, data_2):
    sta_value, p_value = levene(data_1, data_2)
    sta_1, p_value_1 = stats.kstest(data_1, "norm", (np.mean(data_1), np.std(data_2)))
    sta_2, p_value_2 = stats.kstest(data_2, "norm", (np.mean(data_2), np.std(data_2)))
    # plt.hist(data_1, edgecolor='black', normed=True, alpha=0.5)
    # plt.hist(data_2, edgecolor="black", normed=True, alpha=0.5)
    # plt.title("distribution of data")
    # # plt.show()

    if p_value >= 0.05 and p_value_1 >= 0.05 and p_value_2 >= 0.05:
        statistic, pvalue_t = ttest_ind(data_1, data_2)

        # print("| 正态检验: label=1_p_value={:.3f}, label=0_p_value={:.3f}".format(p_value_1, p_value_2))
        # print("| 方差齐性: p_value={:.3f}".format(p_value))
        # print("| T检验: p_value={:.3f}".format(pvalue_t))
        # if pvalue_t < 0.05:
        #     print("| 差异性大！")
        # else:
        #     pass
        return round(pvalue_t, 3)

    else:
        # print("| 正态检验: label=1_p_value={:.3f}, label=0_p_value={:.3f}".format(p_value_1, p_value_2))
        # print("| 方差齐性: p_value={:.3f}".format(p_value))
        stat_num, p_mvalue = mannwhitneyu(data_1, data_2)
        # # print(mannwhitneyu(data_1, data_2))
        #         # print("| u检验: p_value={:.3f}".format(p_mvalue))
        #         # if p_mvalue < 0.05:
        #         #     print("| 差异性大！")
        #         # else:
        #         #     pass
        return round(p_mvalue, 3)



def stats_gender(train, test):
    train_1, train_2 = Counter(train).most_common()
    test_1, test_2 = Counter(test).most_common()

    kf_data = np.array([[np.array(train_1[-1]), np.array(test_1[-1])], [np.array(train_2[-1]), np.array(test_2[-1])]])
    # print(kf_data)
    a, p_value, b, c = chi2_contingency(kf_data)
    print("gender卡方检验: p_value={:.3f}".format(p_value))


def stats_invoved(train, test):
    train_0, train_1 = Counter(train).most_common()
    test_0, test_1 = Counter(test).most_common()

    kf_data_2 = np.array([[np.array(train_0[-1]), np.array(test_0[-1])], [np.array(train_1[-1]), np.array(test_1[-1])]])
    d, p_value_2, f, g = chi2_contingency(kf_data_2)
    print("invoved卡方检验: p_vlaue={:.3f}".format(p_value_2))


def stats_data(data_path):
    pd_data = pd.read_csv(data_path)
    # KF = KFold(n_splits=5, shuffle=False)
    lable = pd_data.loc[:, "label"]
    KF = StratifiedKFold(n_splits=5, shuffle=False, random_state=42)
    for train_index, test_index in KF.split(np.array(pd_data), lable):
        train, test = np.array(pd_data)[train_index], np.array(pd_data)[test_index]
        # print(train.shape, test.shape)
        pd_train = pd.DataFrame(train)
        pd_test = pd.DataFrame(test)

        # 获取数据
        train_age_feature = list(pd_train.loc[:, 2])
        test_age_feature = list(pd_test.loc[:, 2])

        # 进行正态性检验和T检验等
        contin_feature(train_age_feature, test_age_feature)

        train_1_2 = list(pd_train.loc[:, 3])
        test_1_2 = list(pd_test.loc[:, 3])

        train_0_1 = list(pd_train.loc[:, 4])
        test_0_1 = list(pd_test.loc[:, 4])

        stats_gender(train_1_2, test_1_2)
        stats_invoved(train_0_1, test_0_1)
        print("-------------------------------------")


# stats_data(data_path=r"D:\data\numeric_feature.csv")

# 第二次统计
def get_data(data_label_1_path, data_label_0_path):
    data_label_1 = pd.read_csv(data_label_1_path)
    data_label_0 = pd.read_csv(data_label_0_path)
    feature_name_list = list(data_label_1.columns)
    p_value_list = []
    for feature in feature_name_list[2:]:
        data_label_1_feature = data_label_1.loc[:, feature]
        data_label_0_feature = data_label_0.loc[:, feature]

        # print("feature name:{}".format(feature))
        p_value = contin_feature(data_1=data_label_1_feature, data_2=data_label_0_feature)
        p_value_list.append(p_value)

    pd_feature = np.array(feature_name_list)[2:]
    pd_p_value = np.array(p_value_list)
    return pd_feature, pd_p_value





data_label_1_path = r"D:\2019-11-15tongj\label_1.csv"
data_label_0_path = r"D:\2019-11-15tongj\label_0.csv"

case_name, p_value = get_data(data_label_1_path, data_label_0_path)

In [71]:
A = case_name.reshape(-1,1)
B = p_value.reshape(-1, 1)
pd_A = pd.DataFrame(A, columns=["Feature_Name"])
pd_B = pd.DataFrame(B, columns=["P_Value"])
result = pd.concat([pd_A, pd_B], axis=1, )
result.to_csv(r"D:\2019-11-15tongj\result.csv")